## Obtenção e Extração dos dados

In [1]:
# Instalar e importar biblioteca para descompactar arquivo rar

!pip install rarfile

import rarfile

In [2]:
# Baixar arquivo

!wget 'https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar'

--2021-11-09 20:46:10--  https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar
Resolvendo mobileapps.saude.gov.br (mobileapps.saude.gov.br)... 54.233.138.56, 54.232.214.14, 54.233.199.167
Conectando-se a mobileapps.saude.gov.br (mobileapps.saude.gov.br)|54.233.138.56|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 12445782 (12M) [application/x-rar-compressed]
Salvando em: “04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar”

04bd3419b22b9cc5c6e 100%[===================>]  11.87M  1.49MB/s    em 6.9s    

2021-11-09 20:46:18 (1.72 MB/s) - “04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar” salvo [12445782/12445782]



In [3]:
# Extrair arquivos para diretório bases

r = rarfile.RarFile('04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar')
r.extractall('./bases')
r.close()

In [4]:
# Listar arquivos após serem extraídos

!ls -la ./bases/

total 227736
drwxrwxr-x 2 smoke smoke     4096 nov  9 20:46 .
drwxrwxr-x 5 smoke smoke     4096 nov  9 20:46 ..
-rwxrwxr-x 1 smoke smoke 62492959 jul  6 17:28 HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv
-rwxrwxr-x 1 smoke smoke 76520681 jul  6 17:28 HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv
-rwxrwxr-x 1 smoke smoke 91120916 jul  6 17:28 HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
-rwxrwxr-x 1 smoke smoke  3046774 jul  6 17:28 HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv


In [5]:
# Copiar arquivos para container jupyter-spark

!docker cp bases/ jupyter-spark:/root

In [6]:
# Configurar jar do spark para aceitar o formato parquet em tabelas Hive

!curl -O https://repo1.maven.org/maven2/com/twitter/parquet-hadoop-bundle/1.6.0/parquet-hadoop-bundle-1.6.0.jar

!docker cp parquet-hadoop-bundle-1.6.0.jar jupyter-spark:/opt/spark/jars

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2731k  100 2731k    0     0  2047k      0  0:00:01  0:00:01 --:--:-- 2047k


In [7]:
# Verificar arquivos copiados para o container jupyter-spark

!docker exec -it jupyter-spark ls /root/bases

HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv
HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv
HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv


In [8]:
# 1. Enviar os dados para o hdfs

# Criar diretório no hdfs e enviar dados

!docker exec -it jupyter-spark hdfs dfs -mkdir /user/anderson/data/covid

!docker exec -it jupyter-spark hdfs dfs -put /root/bases /user/anderson/data/covid

!docker exec -it jupyter-spark hdfs dfs -ls /user/anderson/data/covid/bases

Found 4 items
-rw-r--r--   2 root supergroup   62492959 2021-11-09 23:49 /user/anderson/data/covid/bases/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv
-rw-r--r--   2 root supergroup   76520681 2021-11-09 23:49 /user/anderson/data/covid/bases/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv
-rw-r--r--   2 root supergroup   91120916 2021-11-09 23:49 /user/anderson/data/covid/bases/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
-rw-r--r--   2 root supergroup    3046774 2021-11-09 23:49 /user/anderson/data/covid/bases/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv
